In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [4]:
# data = pd.read_csv("/content/drive/MyDrive/02dc4105-ca5e-4770-a6fc-d1fdff11fc1c_data.csv").sort_values('created_at')
data = torch.load('datasets/sepc_datasets.pt')
# target = data[['asset_health']]
# 데이터셋의 4번째 컬럼부터 마지막에서 앞으로 6번째 컬럼까지가 유효한 특성들임
# data = data.iloc[:,4:-6]


C:\Users\user\AppData\Local\Temp\ipykernel_4596\4115218575.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('datasets/sepc_datasets.pt')


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/sepc_datasets.pt'

In [ ]:
data = data.reshape(-1, 38)
data.shape

In [ ]:
# target.head()

In [ ]:
# raw_target = target.values
# raw_data = data.values

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Split into train and test datasets
# train_data, test_data, train_labels, test_labels = train_test_split(raw_data, raw_target, test_size=0.2, random_state=21)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_data.shape, type(train_data)

In [ ]:
train_data

In [ ]:
# train_data = torch.tensor(train_data, dtype = torch.float32)
test_data = torch.tensor(test_data, dtype = torch.float32)

In [ ]:
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.fit_transform(test_data)
# autoencoder는 레이블이 필요없음
# train_labels = scaler.fit_transform(train_labels)
# test_labels = scaler.fit_transform(test_labels)

In [ ]:
# Verify the dimensions
print(f"train_data shape: {train_data.shape}, {type(train_data)}")
print(f"test_data shape: {test_data.shape}, {type(test_data)}")

In [ ]:
train_data = torch.tensor(train_data, dtype= torch.float32)
test_data = torch.tensor(test_data, dtype= torch.float32)
train_data

In [ ]:
# # 기준값 설정
# threshold = 1

# # 실수 값을 기준으로 boolean 마스크 생성
# normal_train_mask = train_labels.squeeze() >= threshold  # 정상 데이터: 값이 1 이상
# anomaly_train_mask = train_labels.squeeze() < threshold   # 비정상 데이터: 값이 1 미만

# normal_test_mask = test_labels.squeeze() >= threshold
# anomaly_test_mask = test_labels.squeeze() < threshold

# # 데이터 구분
# normal_train_data = train_data[normal_train_mask]
# anomaly_train_data = train_data[anomaly_train_mask]

# normal_test_data = test_data[normal_test_mask]
# anomaly_test_data = test_data[anomaly_test_mask]

In [ ]:
# # Verify the dimensions
# print(f"normal_train_data shape: {normal_train_data.shape}")
# print(f"normal_test_data shape: {normal_test_data.shape}")
# print(f"anomaly_train_data shape: {anomaly_train_data.shape}")
# print(f"anomaly_test_data shape: {anomaly_test_data.shape}")

In [ ]:
# # Plot data
# plt.grid()
# plt.plot(np.arange(38), normal_train_data[0])
# plt.title("A Normal")
# plt.show()

# plt.grid()
# plt.plot(np.arange(38), anomaly_train_data[0])
# plt.title("An Anomalous")
# plt.show()


In [ ]:
import torch.nn as nn

class AnomalyDetector(nn.Module):
    def __init__(self):
        super(AnomalyDetector, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(38, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 38),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize model
autoencoder = AnomalyDetector()

In [ ]:
# Loss function and optimizer
criterion = nn.L1Loss()  # MAE in PyTorch
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

In [ ]:
autoencoder(train_data)

In [ ]:
# normal_train_data.shape

In [22]:
# Training the autoencoder
# num_epochs = 20
# batch_size = 512

# features_normal_train_data = torch.tensor(normal_train_data)
# train_dataset = torch.utils.data.TensorDataset(features_normal_train_data, features_normal_train_data)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [23]:
# features_test_dataset = torch.tensor(anomaly_train_data)
# test_dataset = torch.utils.data.TensorDataset(features_test_dataset, features_test_dataset)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [29]:
train_dataset = TensorDataset(train_data)

In [30]:
dataloader_train = DataLoader(train_dataset, batch_size = 512, shuffle = True)

In [37]:
for i, x_batch in enumerate(dataloader_train):
    print(x_batch[0])
    if i >= 4:  # Stop after printing 5 batches (0 to 4)
        break

tensor([[0.0044, 0.0019, 0.0057,  ..., 0.0164, 0.0115, 0.1179],
        [0.0390, 0.0738, 0.0114,  ..., 0.0351, 0.0272, 0.0390],
        [0.0590, 0.0693, 0.0139,  ..., 0.0972, 0.0342, 0.0542],
        ...,
        [0.0138, 0.0121, 0.0101,  ..., 0.0031, 0.0151, 0.0340],
        [0.1824, 0.1979, 0.1891,  ..., 0.0526, 0.0790, 0.1607],
        [0.4748, 0.5125, 0.4980,  ..., 0.1429, 0.1446, 0.3552]])
tensor([[0.0024, 0.0014, 0.0056,  ..., 0.0193, 0.0336, 0.0565],
        [0.0392, 0.0562, 0.0161,  ..., 0.0296, 0.0334, 0.1192],
        [0.0604, 0.0328, 0.0098,  ..., 0.0268, 0.0157, 0.1714],
        ...,
        [0.0401, 0.0652, 0.0160,  ..., 0.0345, 0.0292, 0.1173],
        [0.0125, 0.0025, 0.0096,  ..., 0.0286, 0.0244, 0.1423],
        [0.0034, 0.0014, 0.0055,  ..., 0.0161, 0.0254, 0.0706]])
tensor([[0.6555, 0.6420, 0.7401,  ..., 0.3340, 0.2153, 0.5303],
        [0.5004, 0.3874, 0.4677,  ..., 0.1652, 0.1913, 0.2801],
        [0.0666, 0.0498, 0.0485,  ..., 0.0465, 0.0731, 0.1083],
        ...,

In [42]:
num_epochs = 20

for epoch in range(num_epochs):
    autoencoder.train()
    running_loss = 0.0
    for x_batch in dataloader_train:
        # inputs = x_batch
        outputs = autoencoder(x_batch[0])
        loss = criterion(outputs, x_batch[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader_train)}")


Epoch [1/20], Loss: 0.23096254095435143
Epoch [2/20], Loss: 0.22553099691867828
Epoch [3/20], Loss: 0.21994507312774658
Epoch [4/20], Loss: 0.21442070230841637
Epoch [5/20], Loss: 0.20889650657773018
Epoch [6/20], Loss: 0.20327284932136536
Epoch [7/20], Loss: 0.19773338735103607
Epoch [8/20], Loss: 0.19247481226921082
Epoch [9/20], Loss: 0.18714895099401474
Epoch [10/20], Loss: 0.18201838061213493
Epoch [11/20], Loss: 0.17681122198700905
Epoch [12/20], Loss: 0.17176448553800583
Epoch [13/20], Loss: 0.16683697700500488
Epoch [14/20], Loss: 0.16190806403756142
Epoch [15/20], Loss: 0.1571967974305153
Epoch [16/20], Loss: 0.152430709451437
Epoch [17/20], Loss: 0.14790873229503632
Epoch [18/20], Loss: 0.14357347041368484
Epoch [19/20], Loss: 0.1393408440053463
Epoch [20/20], Loss: 0.13512801751494408


In [17]:
for X_train, y_label in train_loader:
  print(X_train.shape)
  print(y_label.shape)
  break

NameError: name 'train_loader' is not defined